In [1]:
import pandas as pd
from Tools import handleTime

In [2]:
def process(inputName, outputName):
    table = pd.read_csv(inputName)
    
    table['time_start'] = pd.to_datetime(
    table.time_window.apply(lambda x: x[1:-1].split(',')[0]))
    
    table['dayofyear'] = table.time_start.apply(lambda x: x.dayofyear)
    table['hour'] = table.time_start.apply(lambda x: x.hour)
    table['minute'] = table.time_start.apply(lambda x: x.minute)
    
    days = {
    'day_of_year': []
    }
    hours = [6, 7]
    for day,df in table.groupby(['dayofyear']):
        days['day_of_year'] += [day]

        for hour in hours:
            for minute in table['minute'].unique():           
                for key, t in table.groupby(['intersection_id', 'tollgate_id']):
                    ind = ((t['minute'] == minute) 
                       & (t['hour'] == hour) 
                       & (t['dayofyear'] == day))

                    t = t[ind]
                    iid, tid = key
                    name = '_'.join(['time' , str(iid) , str(tid) , str(hour) , str(minute)])

                    if(name not in days):
                        days[name] = []

                    if(len(t) == 0):
                        days[name] += [None]
                    elif(len(t) == 1):
                        days[name] += [t.avg_travel_time.values[0]]
                    else:
                        print("Error", len(t)) 
    pd.DataFrame(days).to_csv(outputName, index=None)

In [4]:
inputName = '../training_20min_avg_travel_time.csv'
outputName = 'travel_time_feature.csv'
process(inputName, outputName)

inputName = '../test1_20min_avg_travel_time.csv'
outputName = 'test_travel_time_feature.csv'
process(inputName, outputName)

In [6]:
def process2(inputName, outputName):
    table = pd.read_csv(inputName)
    
    table['time_start'] = pd.to_datetime(
    table.time_window.apply(lambda x: x[1:-1].split(',')[0]))
    
    table['dayofyear'] = table.time_start.apply(lambda x: x.dayofyear)
    table['hour'] = table.time_start.apply(lambda x: x.hour)
    table['minute'] = table.time_start.apply(lambda x: x.minute)
    
    days = {
    'day_of_year': []
    }
    hours = [6, 7]
    for day,df in table.groupby(['dayofyear']):
        days['day_of_year'] += [day]

        for hour in hours:
            for minute in table['minute'].unique():           
                for key, t in table.groupby(['tollgate_id', 'direction']):
                    ind = ((t['minute'] == minute) 
                       & (t['hour'] == hour) 
                       & (t['dayofyear'] == day))

                    t = t[ind]
                    tid, direct = key
                    name = '_'.join(['volume' , str(tid) , str(direct) , str(hour) , str(minute)])

                    if(name not in days):
                        days[name] = []

                    if(len(t) == 0):
                        days[name] += [None]
                    elif(len(t) == 1):
                        days[name] += [t.volume.values[0]]
                    else:
                        print("Error", len(t)) 
    pd.DataFrame(days).to_csv(outputName, index=None)

In [8]:
inputFile = '../training_20min_avg_volume.csv'
outputFile = '../train_volume_feature.csv'

process2(inputFile, outputFile)

inputFile = '../test1_20min_avg_volume.csv'
outputFile = '../test_volume_feature.csv'

process2(inputFile, outputFile)